<a href="https://colab.research.google.com/github/babakansari/DeepLearning/blob/master/Countries_Autocomplete_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 12, 5

Using TensorFlow backend.


In [4]:
import pandas as pd
import io

file_name = 'https://github.com/babakansari/DeepLearning/blob/master/Countries.xlsx?raw=true'

original_df = pd.read_excel(file_name)
original_df.head()


,Name,Ext
0,Afghanistan (AF),NaN
1,Aland Islands (AX),NaN
2,Albania (AL),NaN
3,Algeria (DZ),NaN
4,American Samoa (AS),NaN


In [33]:
df = pd.DataFrame(columns=['Source', 'Target', 'info'])

idx = 0
for index, row in original_df.iterrows():
  name = row['Name']
  nameLen = row['Name'].find('(')

  for i, c in enumerate(name):
    if i == nameLen-2:
      break;
    df.loc[idx] = [name[0:i]+c, name[i+1:nameLen], i]
    idx = idx + 1

df.head(50)

,Source,Target,info
0,A,fghanistan,0
1,Af,ghanistan,1
2,Afg,hanistan,2
3,Afgh,anistan,3
4,Afgha,nistan,4
5,Afghan,istan,5
6,Afghani,stan,6
7,Afghanis,tan,7
8,Afghanist,an,8
9,Afghanista,n,9
